In [2]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import pandas as pd
import re
import requests
import time
from selenium.common.exceptions import StaleElementReferenceException

In [3]:
def getdriver(): #it is the selenium driver to connect the script with the browser
    options = webdriver.ChromeOptions()
    options.add_argument('log-level=3')
    options.add_argument("--window-size=1880x1020")
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-gpu')
    #options.add_argument("--headless")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    options.add_experimental_option("prefs",prefs)
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("test-type")
    options.add_argument("--disable-web-security")
    options.add_argument("--allow-running-insecure-content")
    driver=webdriver.Chrome(options=options, executable_path=ChromeDriverManager().install())
    #     driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'})
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {"source": """
    Object.defineProperty(navigator, 'webdriver', {
        get: () => undefined
    })
    """})
    return driver
driver = getdriver()



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\Pfactorial00110\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
C:\Users\Pfactorial00110\AppData\Local\Temp\ipykernel_14836\4128986626.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(options=options, executable_path=ChromeDriverManager().install())


In [14]:
job_link = []
Category = []
def extract_job_post(url,category): # this function extracts the particular data of the mentioned website
   try: 
      for i in range(0,10):
         url_next=url+str(i) #url to extract
         driver.get(url_next) #connects to the browser
         content = driver.page_source # stores page source in a variable
         soups=BeautifulSoup(content, 'html.parser') 
         soup=soups.findAll('a',href=True) # finds all the a tags in the respective html
         for item in soup:
            if item['href'].startswith('https://www.placementindia.com/job-detail/'): #to extract the particular data which starts with particular words
               job_link.append(item['href']) #to add the extracted data to the empty list
               Category.append(category)
   except:
      pass
   return job_link,Category

In [10]:
job_link, Category = extract_job_post("https://www.placementindia.com/job-search/search.php?seeker_search_keyword=data+science&seeker_search_city=&seeker_search_experience&pageno=", 'Data Science')

In [15]:
job_link, Category = extract_job_post("https://www.placementindia.com/job-search/search.php?seeker_search_keyword=machine+learing'&seeker_search_city=&seeker_search_experience&pageno=", "Machine Learning")

In [17]:
Job_posting_df=pd.DataFrame({"Category":Category,"Job_Posting_link":job_link})

In [13]:
Job_posting_df.to_csv('Data_science_placement_india_links.csv', index=False)

In [18]:
Job_posting_df.to_csv('Machine_learning_placement_india_links.csv', index=False)

In [22]:
DS = pd.read_csv('Data_science_placement_india_links.csv')
ML = pd.read_csv('Machine_learning_placement_india_links.csv')

In [39]:
def fresh_details_extractor(df): # this function extracts the data from the particular job link
    carbs_df = pd.DataFrame()
    for i in range(0,len(df)): #it iterates through the links upto the extracted links length
        try:
          Job_Posting_link=df['Job_Posting_link'][i] #iterates through the links in the dataframe
          Category=df['Category'][i]
          driver.get(Job_Posting_link)
          try:                                                 
            Location= driver.find_element(by=By.XPATH, value='//*[@id="left_content"]/div[1]/div/ul/li[2]/ol/li[2]/span/span').text #extracts the text data from the respective class                                   #/html/body/div[1]/div[2]/main/div[1]/section[1]/div[3]/div[1]/div[1]/div[1]/p[2]
          except:
            Location=None                                      
          try:
            company = driver.find_element(by=By.XPATH, value="//*[@id='filterFx']/div/p/a/span").text #soups1.find(class_ = 'info-org').text
          except:
            company=None
          try:
            title =  driver.find_element(by=By.XPATH, value="//*[@id='filterFx']/div/div/div/p").text #soups1.find(class_ = 'info-position').text
          except:
            title=None
          try:
              salary = driver.find_element(by=By.XPATH, value='//*[@id="left_content"]/div[1]/div/ul/li[2]/div[2]/span').text
          except:
            salary=None
          try:                                                    
            description = driver.find_element(by=By.XPATH, value='//*[@id="left_content"]/div[2]/div/div[1]/div[1]').text #soups1.find(class_='job-description').text 
          except:
            description=None
          try:
            experience = driver.find_element(by=By.XPATH, value='//*[@id="left_content"]/div[1]/div/ul/li[2]/ol/li[1]/span').text
          except:
            experience=None
          try:
            education = driver.find_element(by=By.XPATH, value='/html/body/div/section/div/div[1]/div[1]/div[5]/div[2]/ul/li[3]/span').text #('//div[@class="col"][3]').text #/html/body/div[1]/div[2]/main/div[1]/section[1]/div[3]/div[1]/div[1]/div[3]/p[2]
          except:
            education=None
          try:
            skills = driver.find_element(by=By.XPATH, value='/html/body/div/section/div/div[1]/div[1]/div[5]/div[2]/ul/li[3]/span').text
          except:
            skills = None
          carbs=pd.DataFrame({'Category':Category,'Job_Posting_link':Job_Posting_link,'Job_title':title,'Company_name':company,'Location':Location,'Education':education,'Salary':salary,'Experience':experience,'Skills':skills,'Description':[description]},index=[0])
          carbs_df=carbs_df.append(carbs)
        except Exception as e: # raises an exception if something goes wrong in the time of extraction
          print(e)
    return carbs_df 

In [ ]:
Data_science = fresh_details_extractor(DS)

In [40]:
Machine_learning = fresh_details_extractor(ML)

C:\Users\Pfactorial00110\AppData\Local\Temp\ipykernel_14836\3109740708.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  carbs_df=carbs_df.append(carbs)
C:\Users\Pfactorial00110\AppData\Local\Temp\ipykernel_14836\3109740708.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  carbs_df=carbs_df.append(carbs)
C:\Users\Pfactorial00110\AppData\Local\Temp\ipykernel_14836\3109740708.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  carbs_df=carbs_df.append(carbs)
C:\Users\Pfactorial00110\AppData\Local\Temp\ipykernel_14836\3109740708.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  carbs_df=carbs_df.append(carbs)
C:\Users\Pfactorial0

In [38]:
Data_science.to_csv('placement_india_DS_full_details.csv', index=False)

In [43]:
Machine_learning.to_csv('placement_india_ML_full_details.csv', index=False)

In [37]:
Data_science

,Category,Job_Posting_link,Job_title,Company_name,Location,Education,Salary,Experience,Skills,Description
0,Data Science,https://www.placementindia.com/job-detail/data...,Data Science Training with Placement,Vsoft Technology,"Nagpur, Wadi, Wardha Road, Jaitala, IT Park",None,Not disclosed,0 - 1 yrs,None,Data Science Training\n\nVSoft Technology\n\n#...
0,Data Science,https://www.placementindia.com/job-detail/dot-...,Dot Net Software Developer/ 10 positions/ Job ...,Double HR Consultancy,"Mumbai, Mumbai Harbour",None,"Rs 2,50,000 to 5,00,000 p.a.",2 - 5 yrs,None,Role & Responsibilities\n\nIndividuals are res...
0,Data Science,https://www.placementindia.com/job-detail/dot-...,Dot Net /Angular JA/HTML/PHP Software Develope...,Double HR Consultancy,"Mumbai, Mumbai Harbour",None,"Rs 2,50,000 to 5,00,000 p.a.",2 - 5 yrs,None,Role & Responsibilities\n\nIndividuals are res...
0,Data Science,https://www.placementindia.com/job-detail/opto...,Hiring Optometry/Home Science/Environmental Sc...,Achievers Spot,"Tirupati, Chennai, Madurai, Vellore, Tiruvanna...",None,"Rs 3,25,000 to 4,00,000 p.a.",0 - 1 yrs,None,Identify cases eligible for medical reviews an...
0,Data Science,https://www.placementindia.com/job-detail/nano...,"Vacancies For Nanotechnology,Operation Theater...",Achievers Spot,"Chittoor, Eluru, Nellore, Tirupati, Vijayawada...",None,"Rs 3,25,000 to 4,00,000 p.a.",0 - 1 yrs,None,Identify cases eligible for medical reviews an...
...,...,...,...,...,...,...,...,...,...,...
0,Data Science,https://www.placementindia.com/job-detail/rese...,Research Analyst,YourHRfolks,Bangalore,None,"Rs 1,00,000 to 3,00,000 p.a.",0 - 3 yrs,None,Responsibilities:\n\nUnderstand the objectives...
0,Data Science,https://www.placementindia.com/job-detail/data...,Data Entry Operater,Pioneer Elabs Limited,None,None,None,None,None,Transferring data from paper formats into comp...
0,Data Science,https://www.placementindia.com/job-detail/comp...,Computer Operator,Advancedware Technologies Pvt Ltd,None,None,None,None,None,We are looking for 2 Computer Operator Posts i...
0,Data Science,https://www.placementindia.com/job-detail/assi...,Assistant Manger HR / Manager HR,Sagacious Info System,Bangalore,None,"Rs 30,00,000 to 32,50,000 p.a.",3 - 5 yrs,None,Position: Assistant Manger HR / Manager HR\n\n...


In [41]:
Machine_learning

,Category,Job_Posting_link,Job_title,Company_name,Location,Education,Salary,Experience,Skills,Description
0,Machine Learning,https://www.placementindia.com/job-detail/mach...,MACHINE SHOP HEAD,Titan Consultancy,"Dakshina Kannada, Mangalore",None,Not disclosed,10 - 12 yrs,None,MANGALORE - Coastal area candidates only -\n\n...
0,Machine Learning,https://www.placementindia.com/job-detail/urge...,Urgent Required Of Engineers For Air Condition...,Perfect Solution Group (Spectrum Placement Ser...,"Ghaziabad, Noida",None,"Rs 2,00,000 to 3,25,000 p.a.",1 - 5 yrs,None,Post - Technician (Two Wheeler Refrigerator )\...
0,Machine Learning,https://www.placementindia.com/job-detail/dril...,Drilling Machine Operator,Madhura Industries,None,None,None,None,None,able to use Radial drilling machine to perform...
0,Machine Learning,https://www.placementindia.com/job-detail/pack...,Packaging machine Operator,SGP Food Manufacturers,"Verna, Salcete",None,Not disclosed,1 - 7 yrs,None,Looking for a candidate having good knowledge ...
0,Machine Learning,https://www.placementindia.com/job-detail/cnc-...,CNC oxyfuel cutting and CNC Plasma cutting Mac...,Serntek Engineering Private Limited,None,None,None,None,None,1. Position: CNC Oxyfuel cutting machine opera...
...,...,...,...,...,...,...,...,...,...,...
0,Machine Learning,https://www.placementindia.com/job-detail/cnc-...,CNC Operator/QUALITY CONTROL/QUALITY INSPECTOR,Alliance Consultants,Jamshedpur,None,"Rs 1,00,000 to 1,50,000 p.a.",1 - 2 yrs,None,CNC OPERATOR /QUALITY LAB/ QUALITY INSPECTOR. ...
0,Machine Learning,https://www.placementindia.com/job-detail/prod...,Production Operator,J&V HRSPL,None,None,None,None,None,We are looking for a production worker to join...
0,Machine Learning,https://www.placementindia.com/job-detail/cyli...,Cylindrical Grinding Machine Operator,Universal Engineering And Electrical Company,None,None,None,None,None,Hiring for 2 Cylindrical grinding machine oper...
0,Machine Learning,https://www.placementindia.com/job-detail/mech...,Mechanical Engineer,AJIT RAWAT CONSULTANCY,None,None,None,None,None,Mechanical Engineer\n\nMechanical Engineer Job...
